In [1]:
import altair as alt
import pandas as pd
import numpy as np
import sklearn
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
)

1. loading and filtering of data

In [2]:
data_red = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=";").assign(type = "red")
data_white = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep=";").assign(type = "white")
data_wine = pd.concat([data_red, data_white])
#data_wine = data_wine[(data_wine['quality'] > 3) & (data_wine['quality'] < 8)] 

2. split the data

In [3]:
wine_train, wine_test = train_test_split(data_wine, test_size=0.25, random_state=123)
X_train_raw = wine_train.drop('quality', axis=1)
y_train = wine_train['quality']
X_test_raw = wine_test.drop('quality', axis=1)
y_test = wine_test['quality']

3. preprocessing of data

In [4]:
features_categorical = [
    "type"
]
features_numerical = [
    'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
    'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
    'pH', 'sulphates', 'alcohol'
]
ct = make_column_transformer(
    (StandardScaler(), features_numerical),
    (OneHotEncoder(drop="first"), features_categorical)
)
X_train = ct.fit_transform(X_train_raw)
column_names = (
    features_numerical   
    + ct.named_transformers_["onehotencoder"].get_feature_names_out().tolist()
)
pd.DataFrame(X_train, columns = column_names)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type_white
0,0.453635,0.319504,-0.408391,0.923677,0.094709,0.122813,1.132680,0.270866,-1.112001,-0.737375,0.333868,1.0
1,-0.469247,2.364443,-1.578105,0.005206,1.761534,-1.212652,-1.739088,0.618050,0.820365,-0.260049,0.250535,0.0
2,-1.084502,0.136376,1.242969,-0.934140,2.048917,1.792144,1.080148,-0.846736,-0.737995,-0.532807,-0.666130,1.0
3,1.607238,0.136376,0.486095,-0.829768,0.698215,0.567968,-1.073678,0.608130,1.568378,1.376496,0.333868,0.0
4,0.222915,-0.962397,-0.064358,1.299415,-0.135198,-0.377987,-0.390758,0.766843,-0.176985,-0.600996,-0.832796,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4867,-0.315433,0.685762,-0.270778,4.294883,-0.709965,-0.211053,0.099544,2.734219,-0.987333,-0.737375,0.083868,1.0
4868,0.761263,-0.474054,0.073255,-0.600150,-0.077721,-1.101364,-0.793506,-0.218498,-2.047018,-0.260049,-0.832796,1.0
4869,-0.853781,1.540363,-2.128558,-0.704522,0.008494,-1.435230,-1.809131,0.012958,1.880050,0.217277,0.750533,0.0
4870,0.607449,-0.718226,-0.270778,1.779525,-0.077721,2.905032,1.412853,1.130559,-0.488657,-0.532807,-0.582796,1.0


TODO: performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 

TODO: creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis

4. model

In [5]:
np.random.seed(123)
param_grid = {
    "n_neighbors": range(1, 50, 1),
}
knn_tune = GridSearchCV(
    KNeighborsClassifier(), 
    param_grid, 
    cv=5, 
    n_jobs=-1,
    return_train_score=True
)
knn_model = knn_tune.fit(X_train, y_train)

In [6]:
accuracies = pd.DataFrame(knn_model.cv_results_)

#this one takes a long time to run bc the param grid from above is set to 50. just wait like 4 min and it'll come thorugh!

cv_plot = (alt.Chart(accuracies, title="Accuracy vs Neighbors for Model Including Both Wine Types")
           .transform_fold(["mean_test_score", "mean_train_score"])
           .mark_line()
           .encode(
                x=alt.X(
                    "param_n_neighbors", 
                    title="Neighbors",
                    scale=alt.Scale(zero=False),
                ),
                y=alt.Y(
                    "value:Q", 
                    scale=alt.Scale(zero=False)
                ),
                color='key:N'
            )
)

cv_plot

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [15]:
plots = []

#this one takes even longer to run, just wait and it'll work eventually!

for data in [data_red, data_white]:
    
    wine_train, wine_test = train_test_split(data, test_size=0.25, random_state=123)
    X_train_raw = wine_train.drop(['quality', "type"], axis=1)
    y_train = wine_train['quality']
    X_test_raw = wine_test.drop(['quality', "type"], axis=1)
    y_test = wine_test['quality']

    ct = make_column_transformer(
        (StandardScaler(), features_numerical),
    )
    X_train = ct.fit_transform(X_train_raw)

    pd.DataFrame(X_train, columns = features_numerical)
    np.random.seed(123)

    knn_model = knn_tune.fit(X_train, y_train)

    accuracies = pd.DataFrame(knn_model.cv_results_)

    cv_plot = (alt.Chart(accuracies, title="Accuracy vs Neighbors for Model of :going to add name here: Wine")
               .transform_fold(["mean_test_score", "mean_train_score"])
               .mark_line()
               .encode(
                    x=alt.X(
                        "param_n_neighbors", 
                        title="Neighbors",
                        scale=alt.Scale(zero=False),
                    ),
                    y=alt.Y(
                        "value:Q", 
                        scale=alt.Scale(zero=False)
                    ),
                    color='key:N'
                )
    )

    plots.append(cv_plot)

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


In [16]:
plots[0]

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [17]:
plots[1]

alt.Chart(...)